<a href="https://colab.research.google.com/github/saribasmetehan/Mistral-7B-practices/blob/main/mistral_7b_turkish_finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U transformers bitsandbytes peft datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 53.8 MB/s eta 0:00:00
ERROR: pip's

In [2]:
from transformers import AutoTokenizer

In [3]:
!pip install -q huggingface_hub

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
base_model = "mistralai/Mistral-7B-v0.1"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    padding_side="right",
    add_eos_token=True,
    )

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [7]:
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

In [8]:
import torch

In [9]:
from transformers import BitsAndBytesConfig

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [11]:
from transformers import AutoModelForCausalLM

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [15]:
from datasets import load_dataset

In [29]:
dataset_name = "umarigan/turkiye_finance_qa"
train_dataset = load_dataset(dataset_name, split="train")

In [30]:
train_dataset

Dataset({
    features: ['soru', 'cevap', '__index_level_0__'],
    num_rows: 428
})

In [31]:
!pip install -q pandas

In [32]:
import pandas as pd

In [33]:
train_dataset.to_pandas()

,soru,cevap,__index_level_0__
0,Borsada düşüş ne anlama gelir?,"Borsada düşüş, hisse senedi fiyatlarının genel...",0
1,FED nedir?,"FED, Amerika Birleşik Devletleri'nin merkez ba...",1
2,Enflasyon nedir?,"Enflasyon, genel olarak mal ve hizmetlerin fiy...",2
3,Nasdaq nedir?,"Nasdaq, Amerika Birleşik Devletleri'nde buluna...",3
4,Türev finansal araçlar nelerdir?,"Türev finansal araçlar, gelecekteki bir tariht...",4
...,...,...,...
423,İktisatta talep yasası nedir?,"Talep yasası, diğer faktörler sabit tutulduğun...",440
424,Makroekonomide enflasyon oranının hesaplanması...,"Enflasyon oranı, belirli bir dönemde genel fiy...",441
425,Mikroekonomide tekel piyasa dezavantajları nel...,Tekel piyasanın dezavantajları arasında düşük ...,442
426,İktisatta durgunluk nedir?,"Durgunluk, ekonomik aktivitenin azaldığı, işsi...",443


In [34]:
def generate_promt(sample):
  full_promt = f"""<s>[INST]{sample["soru"]}
                      [/INST]{sample["cevap"]}</s>
                      """
  return {"text":full_promt}

In [35]:
generate_promt(train_dataset[0])

{'text': '<s>[INST]Borsada düşüş ne anlama gelir?\n                      [/INST]Borsada düşüş, hisse senedi fiyatlarının genel olarak değer kaybetmesini ifade eder. Bu durum genellikle ekonomik belirsizlik, şirket performansındaki düşüş veya piyasa beklentilerindeki değişikliklerden kaynaklanabilir.</s>\n                      '}

In [36]:
generetad_train_dataset = train_dataset.map(generate_promt, remove_columns=list(train_dataset.features))

Map:   0%|          | 0/428 [00:00<?, ? examples/s]

In [37]:
generetad_train_dataset

Dataset({
    features: ['text'],
    num_rows: 428
})

In [38]:
from peft import prepare_model_for_kbit_training

In [39]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [40]:
def print_trainable_parameters(model):
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(f"trainable params: {trainable_params} || all params: {all_param} || ratio%: {100*trainable_params/all_param}")

In [41]:
from peft import LoraConfig, get_peft_model

In [42]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj","lm_head"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [43]:
model = get_peft_model(model, lora_config)

In [44]:
print_trainable_parameters(model)

trainable params: 21260288 || all params: 3773331456 || ratio%: 0.5634354746703705


In [45]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): L

In [46]:
from transformers import TrainingArguments

In [47]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.001,
    max_steps=20,
    evaluation_strategy="epoch",
    do_eval=True,
    report_to="none"
    )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [48]:
from trl import SFTTrainer

In [49]:
trainer = SFTTrainer(
    model=model,
    tokenizer = tokenizer,
    args=training_args,
    train_dataset=generetad_train_dataset,
    eval_dataset=generetad_train_dataset,
    peft_config=lora_config,
    dataset_text_field="text"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/u

Map:   0%|          | 0/428 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/428 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [50]:
model.config.use_cache = False

In [51]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,1.470773


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=20, training_loss=1.5823304176330566, metrics={'train_runtime': 739.21, 'train_samples_per_second': 0.108, 'train_steps_per_second': 0.027, 'total_flos': 846245145083904.0, 'train_loss': 1.5823304176330566, 'epoch': 0.18691588785046728})

In [52]:
model_finetuned_finance = "mistral-7b-turkish-finance"

In [53]:
trainer.model.push_to_hub(model_finetuned_finance)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/609M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/saribasmetehan/mistral-7b-turkish-finance/commit/e82d41558e6cae2ce9106515f5492c7801c525d5', commit_message='Upload model', commit_description='', oid='e82d41558e6cae2ce9106515f5492c7801c525d5', pr_url=None, pr_revision=None, pr_num=None)